## 目标:将故障日志中意义相近的故障现象描述进行聚类

### Step 1:将日志中的故障描述提取到List中,并进行分词处理

In [ ]:
import jieba
import xlrd

fileSegWordDonePath ='corpusSegDone.txt'

data = xlrd.open_workbook('log_infos.xls')  # 打开xls文件
table1 = data.sheets()[0] # 打开第一张表

num_cols = table1.col_values(8)

err_info = []

errs = num_cols[1:]

for curr_index in range(len(errs)):
    curr_err = errs[curr_index].encode("utf-8")
    if curr_err != '' and curr_err != '故障描述' :
        err_info.append(curr_err)


fileTrainSeg = []

for i in range(len(err_info)):
    fileTrainSeg.append([' '.join(list(jieba.cut(err_info[i], cut_all=False)))]) #使用jieba进行分词

print len(err_info), "err-infos have been segmented."

with open(fileSegWordDonePath,'wb') as fW:
    for i in range(len(fileTrainSeg)):
        fW.write(fileTrainSeg[i][0].encode('utf-8')) #将分词处理完成的文本写入TXT
        fW.write('\n')

### Step 2:载入停用词表

In [ ]:
#从文件导入停用词表
stpwrdpath = "stop_words.txt"
stpwrd_dic = open(stpwrdpath, 'rb')
stpwrd_content = stpwrd_dic.read()
#将停用词表转换为list  
stpwrdlst = stpwrd_content.splitlines()
stpwrd_dic.close()

### Step ３：对分词后的数据进行向量化处理，这里我选用进行加权词频TF-IDF向量处理

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import sys

reload(sys)
sys.setdefaultencoding('utf-8') # 将系统默认编码格式设为UTF-8

list_source = []

for i in range(len(fileTrainSeg)):
    column_list = fileTrainSeg[i][0].decode('utf-8')
    list_source.append(column_list)  # 加入list_source

#print list_source
with open("list_source.txt",'wb') as fW:
    for i in range(len(list_source)):
        fW.write(list_source[i].encode('utf-8'))
        fW.write('\n')

vectorizer = TfidfVectorizer(stop_words=stpwrdlst, min_df=1)

list_tfidf = []

list_tfidf = vectorizer.fit_transform(list_source) #计算各故障信息的TF-IDF，并将文本转换为词频矩阵

print "TF-IDF process finished."

### step 4: 使用PCA对TF-IDF特征数据进行降维处理

In [ ]:
from sklearn.decomposition import SparsePCA

#将TF-IDF词频矩阵的特征维度降低至适当值
pca = SparsePCA(n_components=50)

final_tfidf = pca.fit_transform(list_tfidf.toarray())

print final_tfidf

### Step 5: 使用K-means对特征矩阵进行聚类处理

In [ ]:
from sklearn.cluster import KMeans

clf = KMeans(n_clusters=20)　　#设定K-means聚类为20个类簇

s = clf.fit(final_tfidf)　　#训练聚类模型

print s

# 20个中心点
print(clf.cluster_centers_)

# 每个样本所属的簇
print(clf.labels_)
i = 1
while i <= len(clf.labels_):
    print i, clf.labels_[i - 1]
    i = i + 1

# 用来评估簇的个数是否合适，距离越小说明簇分的越好，选取临界点的簇个数
print(clf.inertia_)

for i in range(len(list_source)):

    print list_source[i], clf.labels_[i]

    
j=0
with open("result.txt",'wb') as fW:　　#聚类结果存入文本文件中
    while j <= 19:
        fW.write('label')
        fW.write(str(j).encode('utf-8'))
        fW.write('\n')
        for i in range(len(list_source)):
            if clf.labels_[i] == j:
                fW.write(list_source[i].encode('utf-8'))
                fW.write(str(clf.labels_[i]).encode('utf-8'))
                fW.write('\n')
        j = j + 1